<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2e6d93c9c4919056bcf2e896f61d880b1b515b9c45aaeafc78f518ff567006a2
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-29 13:45:42
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.46 C
-------------------
Today PnL: 1.04 L (0.72%)
Current PnL: -18.29 L (-11.98%)
CY Booked + Current PnL: -6.16 L (-4.04%)
-------------------
Total profit:  2.88 L
Total loss:  -21.17 L
-------------------
Total Booked + Current PnL: 20.82 L (16.57%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.31%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.36 L (58.5%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-6.16,76.0,X-LC,4.17,234718.0,24352.0,4436.0,1.19,11.58,1.89,13.68,37.0,5.49,1.63,30.01,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,111.74,66.0,M-SC,1.42,91261.0,-9516.0,9610.0,-2.72,-9.44,10.53,0.09,245.0,-0.99,0.63,19.90,OX40N,NTT,DURABLES
12,BANKINDIA,149.0,-15.53,75.0,H-MC,16.24,216950.0,37142.0,12214.0,-0.02,20.66,5.63,27.45,88.0,3.04,1.50,61.78,XR,NTT,BANKS
42,ITC,452.0,-36.66,64.0,X-LC,1.40,204940.0,4802.0,16088.0,0.29,2.40,7.85,10.44,4.0,0.30,1.42,8.10,X40,NTT,FMCG
2,ABBOTINDIA,35195.0,-9.84,40.0,X-MC,2.76,89145.0,-1533.0,16438.0,-0.18,-1.69,18.44,16.44,101.0,-0.09,0.62,18.14,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-11.76,69.0,X-LC,8.87,317852.0,2010.0,110644.0,9.70,0.64,34.81,35.67,5.0,0.02,2.20,14.58,X40N,ATH,FMCG
62,SAIL,228.00,53.97,68.0,M-MC,10.14,242544.0,17582.0,150062.0,6.58,7.82,61.87,74.53,192.0,0.12,1.68,42.59,XY24,BTT,STEEL
82,VAIBHAVGBL,521.00,64.74,68.0,H-SC,3.85,142692.0,-40083.0,151154.0,5.45,-21.93,105.93,60.77,125.0,-0.27,0.99,27.80,XR,NTT,JEWELLERY
57,RECLTD,446.00,50.38,62.0,M-LC,5.46,210540.0,7370.0,34760.0,3.75,3.63,16.51,20.74,55.0,0.21,1.46,10.74,XY25,NTT,FINANCE
32,HINDZINC,730.22,29.80,52.0,M-LC,11.39,211149.0,6073.0,106504.0,3.01,2.96,50.44,54.89,52.0,0.06,1.46,27.77,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,STARHEALTH,761.00,16.12,50.0,H-SC,12.57,258563.0,-22560.0,155422.0,-3.91,-8.02,60.11,47.26,144.0,-0.15,1.79,38.53,XY24,NTT,INSURANCE
17,CAMS,5211.76,-5.61,48.0,X-SC,0.41,107380.0,5376.0,38549.0,-3.77,5.27,35.90,43.06,122.0,0.14,0.74,23.29,X40N,ATH,MISC
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88885.0,-39785.0,85081.0,-2.73,-30.92,95.72,35.20,219.0,-0.47,0.62,2.02,X40,NTT,FOOTWEAR
79,TTKPRESTIG,770.00,111.74,66.0,M-SC,1.42,91261.0,-9516.0,9610.0,-2.72,-9.44,10.53,0.09,245.0,-0.99,0.63,19.90,OX40N,NTT,DURABLES
11,BANDHANBNK,400.00,-7.85,58.0,H-SC,3.26,230480.0,-48079.0,305524.0,-2.06,-17.26,132.56,92.42,151.0,-0.16,1.60,33.20,XY24,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.0,-29.46,65.0,H-SC,16.16,201474.0,1520.0,95700.0,0.64,0.76,47.50,48.63,115.0,0.02,1.39,12.40,XR,NTT,MISC
87,WIPRO,420.0,-14.70,45.0,M-LC,5.73,150108.0,-837.0,110284.0,-0.11,-0.55,73.47,72.51,53.0,-0.01,1.04,5.43,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,111.74,66.0,M-SC,1.42,91261.0,-9516.0,9610.0,-2.72,-9.44,10.53,0.09,245.0,-0.99,0.63,19.90,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.06,55.0,H-SC,2.75,227034.0,-42633.0,78962.0,1.11,-15.81,34.78,13.47,138.0,-0.54,1.57,15.44,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-22.20,54.0,H-MC,6.76,105184.0,-26651.0,68874.0,-0.16,-20.22,65.48,32.03,98.0,-0.39,0.73,17.82,OX40N,NTT,IT
18,CERA,9475.0,-23.87,44.0,H-SC,2.63,138713.0,-37190.0,79205.0,1.07,-21.14,57.10,23.89,149.0,-0.47,0.96,19.80,OX40N,NTT,CERAMICS
68,SIS,528.0,2042.49,47.0,H-SC,3.15,86458.0,-24574.0,48183.0,-0.85,-22.13,55.73,21.26,156.0,-0.51,0.60,16.81,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.0,-15.53,75.0,H-MC,16.24,216950.0,37142.0,12214.0,-0.02,20.66,5.63,27.45,88.0,3.04,1.5,61.78,XR,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,53.97,68.0,M-MC,10.14,242544.0,17582.0,150062.0,6.58,7.82,61.87,74.53,192.0,0.12,1.68,42.59,XY24,BTT,STEEL
57,RECLTD,446.00,50.38,62.0,M-LC,5.46,210540.0,7370.0,34760.0,3.75,3.63,16.51,20.74,55.0,0.21,1.46,10.74,XY25,NTT,FINANCE
32,HINDZINC,730.22,29.80,52.0,M-LC,11.39,211149.0,6073.0,106504.0,3.01,2.96,50.44,54.89,52.0,0.06,1.46,27.77,X5K,ATH,METALS
47,KPIGREEN,731.05,10.94,72.0,H-SC,5.37,132136.0,6839.0,52088.0,0.33,5.46,39.42,47.03,141.0,0.13,0.91,65.45,MH,ATH,POWER
80,UNIONBANK,163.00,-8.53,64.0,M-LC,9.41,166013.0,25173.0,19324.0,-0.72,17.87,11.64,31.59,66.0,1.30,1.15,49.10,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.00,-15.53,75.0,H-MC,16.24,216950.0,37142.0,12214.0,-0.02,20.66,5.63,27.45,88.0,3.04,1.50,61.78,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-37.26,66.0,M-SC,3.39,130014.0,15216.0,78827.0,2.32,13.25,60.63,81.92,223.0,0.19,0.90,53.92,XR,NTT,DURABLES
47,KPIGREEN,731.05,10.94,72.0,H-SC,5.37,132136.0,6839.0,52088.0,0.33,5.46,39.42,47.03,141.0,0.13,0.91,65.45,MH,ATH,POWER
32,HINDZINC,730.22,29.80,52.0,M-LC,11.39,211149.0,6073.0,106504.0,3.01,2.96,50.44,54.89,52.0,0.06,1.46,27.77,X5K,ATH,METALS
37,INDIAMART,4810.62,-52.96,58.0,H-SC,6.80,131345.0,8009.0,123622.0,-0.09,6.49,94.12,106.72,119.0,0.06,0.91,31.39,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,18.69,149374.0,-127149.0,237221.0,0.35,-45.98,158.81,39.81,54.0,-0.54,1.03,5.28,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88885.0,-39785.0,85081.0,-2.73,-30.92,95.72,35.20,219.0,-0.47,0.62,2.02,X40,NTT,FOOTWEAR
45,JSWINFRA,342.00,-20.48,37.0,X-MC,5.33,199631.0,-716.0,31222.0,0.27,-0.36,15.64,15.23,178.0,-0.02,1.38,31.78,X40N,NTT,REALTY
24,DMART,5391.45,-16.82,38.0,X-LC,9.89,101731.0,-855.0,27661.0,0.42,-0.83,27.19,26.13,38.0,-0.03,0.70,24.53,X40N,ATH,FMCG
2,ABBOTINDIA,35195.00,-9.84,40.0,X-MC,2.76,89145.0,-1533.0,16438.0,-0.18,-1.69,18.44,16.44,101.0,-0.09,0.62,18.14,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-2.40,49.0,H-LC,1.06,157000.0,-29095.0,76569.0,0.97,-15.63,48.77,25.51,15.0,-0.38,1.09,15.70,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,64.0,X-LC,1.40,204940.0,4802.0,16088.0,0.29,2.40,7.85,10.44,4.0,0.30,1.42,8.10,X40,NTT,FMCG
53,PGHH,17907.65,-30.28,40.0,X-MC,2.58,201945.0,1125.0,66662.0,0.49,0.56,33.01,33.76,80.0,0.02,1.40,5.79,X40,ATH,FMCG
29,HAVELLS,2069.16,-6.89,42.0,X-MC,3.08,310926.0,-17065.0,123593.0,-0.80,-5.20,39.75,32.48,92.0,-0.14,2.15,5.63,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,1.30,53.0,X-MC,3.90,212415.0,20673.0,17078.0,-0.86,10.78,8.04,19.69,99.0,1.21,1.47,18.31,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88885.0,-39785.0,85081.0,-2.73,-30.92,95.72,35.20,219.0,-0.47,0.62,2.02,X40,NTT,FOOTWEAR
52,PAGEIND,51769.93,-27.18,41.0,X-MC,6.80,82210.0,-1300.0,21333.0,0.46,-1.56,25.95,23.98,82.0,-0.06,0.57,4.17,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-14.06,49.0,X-MC,5.92,181138.0,-19368.0,116526.0,1.32,-9.66,64.33,48.45,90.0,-0.17,1.25,4.45,X40N,ATH,FINANCE
21,COLPAL,3767.14,-3.45,49.0,X-MC,7.87,225180.0,-38185.0,151524.0,0.86,-14.50,67.29,43.04,84.0,-0.25,1.56,4.54,XY25,ATH,FMCG
3,ACC,3906.00,-37.20,57.0,X-MC,2.54,187300.0,-50651.0,203295.0,0.66,-21.29,108.54,64.15,174.0,-0.25,1.30,5.07,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-44.90,52.0,X-SC,38.20,54498.0,-10508.0,161434.0,1.06,-16.16,296.22,232.18,198.0,-0.07,0.38,6.30,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.18,41.0,X-MC,6.80,82210.0,-1300.0,21333.0,0.46,-1.56,25.95,23.98,82.0,-0.06,0.57,4.17,X40,ATH,APPARELS
58,RELAXO,1176.00,-40.07,42.0,X-SC,4.68,81677.0,-63483.0,139414.0,-0.11,-43.73,170.69,52.31,136.0,-0.46,0.57,9.33,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.84,40.0,X-MC,2.76,89145.0,-1533.0,16438.0,-0.18,-1.69,18.44,16.44,101.0,-0.09,0.62,18.14,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88885.0,-39785.0,85081.0,-2.73,-30.92,95.72,35.20,219.0,-0.47,0.62,2.02,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-27.72,55.0,X-LC,11.30,287772.0,-58184.0,124893.0,0.11,-16.82,43.40,19.28,1.0,-0.47,1.99,6.38,X40,ATH,IT
40,INFY,2275.00,-17.09,61.0,X-LC,6.14,322503.0,9615.0,162058.0,0.93,3.07,50.25,54.87,3.0,0.06,2.23,11.61,X40,BTT,IT
42,ITC,452.00,-36.66,64.0,X-LC,1.40,204940.0,4802.0,16088.0,0.29,2.40,7.85,10.44,4.0,0.30,1.42,8.10,X40,NTT,FMCG
84,VBL,671.64,-11.76,69.0,X-LC,8.87,317852.0,2010.0,110644.0,9.70,0.64,34.81,35.67,5.0,0.02,2.20,14.58,X40N,ATH,FMCG
1,ABB,7934.00,-39.48,56.0,H-LC,5.80,253080.0,-8539.0,127755.0,1.87,-3.26,50.48,45.57,7.0,-0.07,1.75,7.69,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7127.78,64.0,L-SC,7.62,79317.0,-14493.0,91111.0,-0.00,-15.45,114.87,81.67,269.0,-0.16,0.55,53.60,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,50.0,L-SC,27.94,81619.0,-31930.0,71980.0,-0.19,-28.12,88.19,35.27,268.0,-0.44,0.57,99.77,XR,NTT,HOTELS
50,MASFIN,398.61,-20.05,46.0,H-SC,9.15,90795.0,-7185.0,28791.0,0.31,-7.33,31.71,22.05,152.0,-0.25,0.63,31.98,XR,ATH,FINANCE
79,TTKPRESTIG,770.00,111.74,66.0,M-SC,1.42,91261.0,-9516.0,9610.0,-2.72,-9.44,10.53,0.09,245.0,-0.99,0.63,19.90,OX40N,NTT,DURABLES
86,WHIRLPOOL,2270.00,-37.26,66.0,M-SC,3.39,130014.0,15216.0,78827.0,2.32,13.25,60.63,81.92,223.0,0.19,0.90,53.92,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,-196.59,75.0,M-SC,29.67,166626.0,17406.0,126769.0,-1.13,11.66,76.08,96.62,208.0,0.14,1.15,80.33,XY25,NTT,FINANCE
69,SONACOMS,806.63,-32.06,71.0,M-SC,7.97,84578.0,-16581.0,56583.0,-0.22,-16.39,66.90,39.54,202.0,-0.29,0.59,19.26,AR,ATH,AUTO
12,BANKINDIA,149.00,-15.53,75.0,H-MC,16.24,216950.0,37142.0,12214.0,-0.02,20.66,5.63,27.45,88.0,3.04,1.50,61.78,XR,NTT,BANKS
47,KPIGREEN,731.05,10.94,72.0,H-SC,5.37,132136.0,6839.0,52088.0,0.33,5.46,39.42,47.03,141.0,0.13,0.91,65.45,MH,ATH,POWER
5,ANGELONE,3033.00,16.33,63.0,X-SC,5.43,205656.0,14650.0,43044.0,-1.18,7.67,20.93,30.21,157.0,0.34,1.42,30.15,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.57
1,25,44.01
2,50,74.92


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.01
LC    32.43
MC    24.56
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.02
X40      16.21
XY25     12.07
XR       11.44
X40N     10.68
AR        7.85
OX40N     6.52
X200      1.76
X5K       1.46
SR        1.08
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.39
X-LC    21.72
X-MC    18.68
M-SC    12.67
M-LC     5.11
H-LC     4.60
X-SC     4.50
H-MC     3.88
M-MC     1.68
L-SC     1.45
L-LC     1.00
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.56,-2.45,35.88
IT,12.50,-16.35,78.76
FINANCE,12.09,-8.18,57.22
MISC,6.84,-16.27,80.70
ELECTRICAL,5.76,-8.62,47.13
BANKS,5.66,-4.84,59.06
PAINTS,5.50,-16.14,33.45
INSURANCE,3.77,-1.47,39.31
AUTO,2.76,-44.24,104.44


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3237917.0,22
XR,1234536.0,14
AR,1156406.0,9
X40,814879.0,12
X40N,615334.0,9
OX40N,542448.0,9
XY25,440538.0,8
SR,265834.0,2
X5K,106504.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3311278.0,24
M-SC,1214429.0,15
X-MC,1178584.0,14
X-LC,999614.0,13
X-SC,531670.0,6
H-LC,273743.0,3
M-LC,270872.0,4
H-MC,256694.0,3
L-SC,252095.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1155328.0      6
           AR         829226.0      5
           XR         766520.0      7
M-SC       XY24       696904.0      6
X-MC       XY24       575389.0      4
X-LC       X40        442953.0      5
           XY24       303870.0      2
X-MC       X40        286845.0      6
X-SC       X40N       285155.0      4
H-SC       SR         265834.0      2
           OX40N      242282.0      3
H-LC       AR         204324.0      2
X-LC       X40N       182431.0      3
H-MC       XY24       175606.0      1
X-MC       XY25       168602.0      2
L-SC       XR         163091.0      2
X-SC       XY24       161434.0      1
M-MC       XY24       150062.0      1
X-MC       X40N       147748.0      2
M-SC       OX40N      142288.0      4
           XY25       126769.0      1
           XR         125612.0      2
           AR         122856.0      2
M-LC       XR         110284.0      1
           X5K        106504.0      1
L-SC       OX40N       89004.0      1
X-SC       X40         85081.0      1
X-LC       XY25        70360.0      3
H-LC       X200        69419.0      1
H-MC       OX40N       68874.0      1
L-MC       XR          56815.0      1
H-SC       MH          52088.0      1
L-LC       XY25        40047.0      1
M-LC       XY25        34760.0      1
           XY24        19324.0      1
H-MC       XR          12214.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 51.0 seconds
